In [2]:
from gurobipy import *
from typing import Dict
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import SETTING
import RL
import importlib
import tool_func
from update import *

num_vehicle = 20
num_order = 15
num_city = 8
TIME = 100
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles ={}
speed = 20
cancel_penalty = 300
battery_consume = 10
battery_add = 300
speed = 200
matrix = np.tile(row, (num_vehicle, 1))

# 初始化
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city,0.3, (20,100))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order ={}
for time in range(TIME):
    group = [[],[]]
    for vehicle in Vehicles.values():
        if vehicle.whether_city:
            group[0].append(vehicle.id)
            
        else:
            group[1].append(vehicle.id)
    
    Orders = order_generator(num_order, time, num_city-1,CAPACITY,G)
    for order in Orders.values():
        orders_unmatched[order.id] = order
        Total_order[order.id] = order

    orders_virtual = orders_unmatched
    if time == 0:
        city_node = city_node_generator(G,orders_virtual,Vehicles,orders_unmatched)
    # order_virtual = RL.allocate(orders_unmatched) #此处强化学习
    if len(group[0]) != 0:
        temp_Lower_Layer = Lower_Layer(G,city_node ,Vehicles, orders_unmatched ,name,group,time)
        temp_Lower_Layer.get_decision()
        temp_Lower_Layer.constrain_1()
        temp_Lower_Layer.constrain_2()
        temp_Lower_Layer.constrain_3()
        temp_Lower_Layer.constrain_4()
        temp_Lower_Layer.constrain_5()
        temp_Lower_Layer.model.setParam('OutputFlag', 0)
        total_penalty = cancel_penalty*order_canceled
        temp_Lower_Layer.set_objective(matrix,total_penalty)
        temp_Lower_Layer.model.optimize()
        if temp_Lower_Layer.model.status == GRB.OPTIMAL:
            # 打印变量的最优值
            print(f"{time}次，Optimal solution:")
            i = 0
            with open(f"output_{time}.txt", "w") as file:
                for v in temp_Lower_Layer.model.getVars():
                    if v.x == 1:
                        file.write(f"{v.varName} = {v.x}\n")  # 将结果写入文件
                # 打印目标函数值
            print("Objective value:", temp_Lower_Layer.model.objVal)
            
            
        else:

            temp_Lower_Layer.model.computeIIS()
            temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
            print(f"{time}次，No optimal solution found.")
            self_update(Vehicles,G)
        _,var_order = temp_Lower_Layer.get_decision()
        update_var(temp_Lower_Layer, Vehicles,orders_unmatched)
        vehicle_in_city = update_vehicle(Vehicles,battery_consume, battery_add, speed, G)
        order_canceled = update_order(orders_unmatched,time,speed)
        #print(f"{len(orders_unmatched)}个订单未被匹配")
        #print(f"{vehicle_in_city}辆汽车可调度")
       
    else:
        self_update(Vehicles,G)
        print(f"{time}次，没有车")
        vehicle_in_city = update_vehicle(Vehicles,battery_consume, battery_add, speed, G)
        order_canceled = update_order(orders_unmatched,time,speed)
        #print(f"{len(orders_unmatched)}个订单未被匹配")
        #print(f"{vehicle_in_city}辆汽车可调度")

0次，Optimal solution:
Objective value: 123384.0
1次，Optimal solution:
Objective value: 129172.0
2次，Optimal solution:
Objective value: 59340.0
3次，Optimal solution:
Objective value: 43547.0
4次，没有车
5次，Optimal solution:
Objective value: 36538.0
6次，Optimal solution:
Objective value: 30427.0
7次，没有车
8次，没有车
9次，没有车
10次，没有车
11次，没有车
12次，没有车
13次，没有车
14次，没有车
15次，没有车
16次，没有车
17次，没有车
18次，没有车
19次，没有车
20次，没有车
21次，没有车
22次，没有车
23次，没有车
24次，没有车
25次，没有车
26次，没有车
27次，没有车
28次，没有车
29次，没有车
30次，没有车
31次，没有车
32次，没有车
33次，没有车
34次，没有车
35次，没有车
36次，没有车
37次，没有车
38次，没有车
39次，没有车
40次，没有车
41次，没有车
42次，没有车
43次，没有车
44次，没有车
45次，没有车
46次，没有车
47次，没有车
48次，没有车
49次，没有车
50次，没有车
51次，没有车
52次，没有车
53次，没有车
54次，没有车
55次，没有车
56次，没有车
57次，没有车
58次，没有车
59次，没有车
60次，没有车
61次，没有车
62次，没有车
63次，没有车
64次，没有车
65次，没有车
66次，没有车
67次，没有车
68次，没有车
69次，没有车
70次，没有车
71次，没有车
72次，没有车
73次，没有车
74次，没有车
75次，没有车
76次，没有车
77次，没有车
78次，没有车
79次，没有车
80次，没有车
81次，没有车
82次，没有车
83次，没有车
84次，没有车
85次，没有车
86次，没有车
87次，没有车
88次，没有车
89次，没有车
90次，没有车
91次，没有车
92次，没有车
93次，没有车
94次，没有车
95次，没有车
96次，没有

In [1]:
for vehicle in Vehicles.values():
    #if vehicle.id == 0:
        #print(vehicle)
        print(vehicle.get_history_decisions())

NameError: name 'Vehicles' is not defined

In [3]:
for order in Total_order.values():
    print(order)

Order(id=0, passengers=4, departure=6, destination=4,virtual_departure=6, matched_vehicle_id=0,distance=57,battery=577.551697218503, status=Matched)
Order(id=1, passengers=3, departure=4, destination=2,virtual_departure=4, matched_vehicle_id=1,distance=64,battery=645.6767017636662, status=Matched)
Order(id=2, passengers=3, departure=4, destination=6,virtual_departure=4, matched_vehicle_id=1,distance=57,battery=571.4146641622207, status=Matched)
Order(id=3, passengers=6, departure=4, destination=1,virtual_departure=4, matched_vehicle_id=2,distance=118,battery=1180.71852272161, status=Matched)
Order(id=4, passengers=3, departure=2, destination=1,virtual_departure=2, matched_vehicle_id=0,distance=54,battery=542.3797371972581, status=Matched)
Order(id=5, passengers=6, departure=4, destination=2,virtual_departure=4, matched_vehicle_id=3,distance=64,battery=641.8694520275466, status=Matched)
Order(id=6, passengers=1, departure=3, destination=4,virtual_departure=3, matched_vehicle_id=1,distan

In [7]:
for vehicle in Vehicles.values():
    if vehicle.get_orders():
        for order in vehicle.get_orders():
            if order.matched_vehicle_id == 8:
                print(vehicle.id)
                

In [8]:
for order in Total_order.values():
    if order.matched_vehicle_id == 8:
        print(order)

Order(id=13, passengers=2, departure=5, destination=2,virtual_departure=5, matched_vehicle_id=8,battery=287.9966258615341, status=Matched)
Order(id=15, passengers=5, departure=4, destination=2,virtual_departure=4, matched_vehicle_id=8,battery=849.64585838219, status=Matched)


In [3]:
for vehicle in Vehicles.values():
    #if vehicle.id == 0:
        print(vehicle)
        #print(vehicle.get_history_decisions())

Vehicle(id=0, time=100, into_city=4, intercity=2, passenger=6, decision=3, battery=13146.289404829768, matched_order=3)
Vehicle(id=1, time=100, into_city=6, intercity=0, passenger=6, decision=3, battery=16927.32952300423, matched_order=1)
Vehicle(id=2, time=100, into_city=3, intercity=2, passenger=4, decision=3, battery=1711.6264989390825, matched_order=1)
Vehicle(id=3, time=100, into_city=3, intercity=2, passenger=2, decision=3, battery=5379.198192799372, matched_order=1)
Vehicle(id=4, time=100, into_city=2, intercity=3, passenger=7, decision=3, battery=10908.07589746097, matched_order=3)
Vehicle(id=5, time=100, into_city=1, intercity=2, passenger=4, decision=3, battery=4067.5100023358427, matched_order=1)
Vehicle(id=6, time=100, into_city=2, intercity=4, passenger=7, decision=3, battery=4591.004037749382, matched_order=2)
Vehicle(id=7, time=100, into_city=6, intercity=4, passenger=7, decision=3, battery=16934.89234412165, matched_order=1)
Vehicle(id=8, time=100, into_city=4, intercit

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import SETTING
import RL
import importlib
import tool_func
from update import *

num_vehicle = 20
num_order = 15
num_city = 8
TIME = 100
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles = {}
speed = 20
cancel_penalty = 300
battery_consume = 10
battery_add = 300
speed = 200
matrix = np.tile(row, (num_vehicle, 1))

# 初始化
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city, 0.3, (20, 100))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order = {}

# 记录所有时间点车辆信息
all_vehicle_records = []

for time in range(TIME):
    group = [[], []]
    
    # 记录每个时刻的车辆信息
    vehicle_records = []

    for vehicle in Vehicles.values():
        # 将车辆信息存入字典
        vehicle_records.append({
            "id": vehicle.id,
            "time": vehicle.time,
            "into_city": vehicle.into_city,
            "intercity": vehicle.intercity,
            "passenger": vehicle.get_capacity(),
            "decision": vehicle.decision,
            "battery": vehicle.battery,
            "whether_city":vehicle.whether_city,
            "matched_order": vehicle.orders,
        })

        # 通过条件分组
        if vehicle.whether_city:
            group[0].append(vehicle.id)
        else:
            group[1].append(vehicle.id)

    # 将当前时刻的车辆记录添加到所有记录中
    all_vehicle_records.append(vehicle_records)

    # 生成订单
    Orders = order_generator(num_order, time, num_city-1, CAPACITY, G)
    for order in Orders.values():
        orders_unmatched[order.id] = order
        Total_order[order.id] = order

    orders_virtual = orders_unmatched
    if time == 0:
        city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
    
    if len(group[0]) != 0:
        temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
        temp_Lower_Layer.get_decision()
        temp_Lower_Layer.constrain_1()
        temp_Lower_Layer.constrain_2()
        temp_Lower_Layer.constrain_3()
        temp_Lower_Layer.constrain_4()
        temp_Lower_Layer.constrain_5()
        temp_Lower_Layer.model.setParam('OutputFlag', 0)
        total_penalty = cancel_penalty * order_canceled
        temp_Lower_Layer.set_objective(matrix, total_penalty)
        temp_Lower_Layer.model.optimize()

        if temp_Lower_Layer.model.status == GRB.OPTIMAL:
            # 打印最优解
            print(f"{time}次，Optimal solution:")
            i = 0
            with open(f"output_{time}.txt", "w") as file:
                for v in temp_Lower_Layer.model.getVars():
                    if v.x == 1:
                        file.write(f"{v.varName} = {v.x}\n")  # 将结果写入文件
            print("Objective value:", temp_Lower_Layer.model.objVal)
            
        else:
            temp_Lower_Layer.model.computeIIS()
            temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
            print(f"{time}次，No optimal solution found.")
            self_update(Vehicles, G)
        
        _, var_order = temp_Lower_Layer.get_decision()
        update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
        vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
        order_canceled = update_order(orders_unmatched, time, speed)
       
    else:
        self_update(Vehicles, G)
        print(f"{time}次，没有车")
        vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
        order_canceled = update_order(orders_unmatched, time, speed)

# 将记录保存为 CSV 文件
csv_file = "vehicle_records.csv"
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    # 写入CSV文件的列名
    fieldnames = ["id", "time", "into_city", "intercity", "passenger", "decision", "battery", "whether_city","matched_order"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    
    # 遍历所有时刻的车辆记录
    for vehicle_records in all_vehicle_records:
        writer.writerows(vehicle_records)

print(f"Vehicle information saved to {csv_file}")


0次，Optimal solution:
Objective value: 118085.0
1次，Optimal solution:
Objective value: 113094.0
2次，Optimal solution:
Objective value: 26470.0
3次，Optimal solution:
Objective value: 94457.0
4次，Optimal solution:
Objective value: 55340.0
5次，Optimal solution:
Objective value: 97701.0
6次，Optimal solution:
Objective value: 34727.0
7次，Optimal solution:
Objective value: 27931.0
8次，Optimal solution:
Objective value: 62704.0
9次，没有车
10次，Optimal solution:
Objective value: 47096.0
11次，没有车
12次，没有车
13次，Optimal solution:
Objective value: 67856.0
14次，没有车
15次，没有车
16次，没有车
17次，没有车
18次，没有车
19次，没有车
20次，没有车
21次，没有车
22次，没有车
23次，没有车
24次，没有车
25次，没有车
26次，没有车
27次，没有车
28次，没有车
29次，没有车
30次，没有车
31次，没有车
32次，没有车
33次，没有车
34次，没有车
35次，没有车
36次，没有车
37次，没有车
38次，没有车
39次，没有车
40次，没有车
41次，没有车
42次，没有车
43次，没有车
44次，没有车
45次，没有车
46次，没有车
47次，没有车
48次，没有车
49次，没有车
50次，没有车
51次，没有车
52次，没有车
53次，没有车
54次，没有车
55次，没有车
56次，没有车
57次，没有车
58次，没有车
59次，没有车
60次，没有车
61次，没有车
62次，没有车
63次，没有车
64次，没有车
65次，没有车
66次，没有车
67次，没有车
68次，没有车
69次，没有车
70次，没有车
71次，没有车
72次

PermissionError: [Errno 13] Permission denied: 'vehicle_records.csv'